# Check Python version and upgrading

In [5]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
!pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable


In [6]:
import sys
import os

In [7]:
# !pip install torch
# !pip install torchvision
# !conda update -n base -c defaults conda
# !conda install -y pytorch torchvision -c pytorch

# Installations

In [8]:
#KILT
%rm -rf KILT
!git clone https://github.com/facebookresearch/KILT.git
%cd KILT
!pip install -r requirements.txt
!python setup.py install
sys.path.append(os.getcwd())
%cd ..

#GENRE

%rm -rf GENRE
!git clone https://github.com/facebookresearch/GENRE.git
%cd GENRE
!pip install ./
!python ./setup.py build develop install
!pip install sentencepiece marisa_trie
sys.path.append(os.getcwd())
%cd ..

# fairseq

%rm -rf fairseq
!git clone --branch fixing_prefix_allowed_tokens_fn https://github.com/nicola-decao/fairseq
%cd fairseq
!sed -i -e '26,27d' fairseq/registry.py
!pip install --editable ./
!python setup.py build develop
!python setup.py install
sys.path.append(os.getcwd())
%cd ..


# !pip install transformers
# !pip install omegaconf
# !pip install unidecode
# !pip install requests
!pip install kilt
# !pip install fairseq
# !pip install hydra-core
# !pip install bitarray
# !pip install gdown
# !pip install --upgrade gdown

#####
%cd fairseq
!pip install --editable ./
%cd ..

Cloning into 'KILT'...
remote: Enumerating objects: 401, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 401 (delta 107), reused 92 (delta 92), pack-reused 251
Receiving objects: 100% (401/401), 829.83 KiB | 3.28 MiB/s, done.
Resolving deltas: 100% (224/224), done.
/home/petrakov/mGENRE_MEL/KILT/KILT
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Traceback (most recent call last):
  File "setup.py", line 8, in <module>
    import setuptools
ImportError: No module named setuptools
/home/petrakov/mGENRE_MEL/KILT
Cloning into 'GENRE'...
remote: Enumerating objects: 454, done.
remote: Counting objects: 100% (170/170), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 454 (delta 112), reused 104 (delta 80), pack-reused 284
Receiving objects: 100% (454/454), 10.99 MiB | 7.59 MiB/

In [9]:
!pip install kilt

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement kilt (from versions: none)
ERROR: No matching distribution found for kilt


## import libraries

In [24]:
import gdown
import json
import pickle
import torch
from wikidata.client import Client

## Upload data and model (comment this cell if already downloaded)

In [12]:
# pretrained model
! rm -rf fairseq_multilingual_entity_disambiguation*
! wget https://dl.fbaipublicfiles.com/GENRE/fairseq_multilingual_entity_disambiguation.tar.gz
! tar -xvf fairseq_multilingual_entity_disambiguation.tar.gz

# data
! wget https://dl.fbaipublicfiles.com/GENRE/lang_title2wikidataID-normalized_with_redirect.pkl
! wget http://dl.fbaipublicfiles.com/GENRE/titles_lang_all105_marisa_trie_with_redirect.pkl

--2022-06-27 17:52:35--  https://dl.fbaipublicfiles.com/GENRE/fairseq_multilingual_entity_disambiguation.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 188.114.98.160, 2a06:98c1:3123:a000::
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|188.114.98.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2057741836 (1.9G) [application/gzip]
Saving to: ‘fairseq_multilingual_entity_disambiguation.tar.gz’

fairseq_multilingua 100%[===================>]   1.92G  16.4MB/s    in 1m 48s  

2022-06-27 17:54:24 (18.2 MB/s) - ‘fairseq_multilingual_entity_disambiguation.tar.gz’ saved [2057741836/2057741836]

fairseq_multilingual_entity_disambiguation/
fairseq_multilingual_entity_disambiguation/model.pt
fairseq_multilingual_entity_disambiguation/dict.source.txt
fairseq_multilingual_entity_disambiguation/dict.target.txt
fairseq_multilingual_entity_disambiguation/spm_256000.model
fairseq_multilingual_entity_disambiguation/spm_256000.vocab
--202

## Install test data

In [13]:
id_ = "1QpLVXGahVjAoEq8TrE70aKtMLJCNeASh"
gdown.download(id=id_, output="mentions_test.json", quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1QpLVXGahVjAoEq8TrE70aKtMLJCNeASh
To: /home/petrakov/mGENRE_MEL/KILT/mentions_test.json
100%|██████████| 65.1M/65.1M [00:00<00:00, 72.8MB/s]


'mentions_test.json'

In [14]:
with open("mentions_test.json") as f:
    test_set = json.load(f)

In [15]:
#https://drive.google.com/file/d/1EfRYS0V3C0K7u_4SuP6DcEi31T_7dpq_/view?usp=sharing

id__ = "1EfRYS0V3C0K7u_4SuP6DcEi31T_7dpq_"
gdown.download(id=id__, output="fairseq_model.py", quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1EfRYS0V3C0K7u_4SuP6DcEi31T_7dpq_
To: /home/petrakov/mGENRE_MEL/KILT/fairseq_model.py
100%|██████████| 5.10k/5.10k [00:00<00:00, 3.68MB/s]


'fairseq_model.py'

In [16]:
%mv fairseq_model.py GENRE/genre

# Multilingual Autoregressive Entity Linking

![](s.png)

In [17]:
from GENRE.genre.trie import Trie, MarisaTrie
from GENRE.genre.fairseq_model import mGENRE

In [18]:
with open("lang_title2wikidataID-normalized_with_redirect.pkl", "rb") as f:
    lang_title2wikidataID = pickle.load(f)
  
# with open("titles_lang_all105_trie_with_redirect.pkl", "rb") as f:
#     trie = Trie.load_from_dict(pickle.load(f))

with open("titles_lang_all105_marisa_trie_with_redirect.pkl", "rb") as f:
    trie = pickle.load(f)

In [37]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [38]:
# generate Wikipedia titles and language IDs
model = mGENRE.from_pretrained("fairseq_multilingual_entity_disambiguation").eval()
model.to(device)

GENREHubInterface(
  (models): ModuleList(
    (0): BARTModel(
      (encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(256001, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
        (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0): TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (dropout_module): FairseqDro

In [39]:
sentences = ["[START] The founder of the theory of relativity [END] received the Nobel Prize."]
model.sample(
    sentences,
    beam = 5,
    prefix_allowed_tokens_fn=lambda batch_id, sent: [
        e for e in trie.get(sent.tolist())
        if e < len(model.task.target_dictionary)
    ],
    text_to_id=lambda x: max(lang_title2wikidataID[tuple(reversed(x.split(" >> ")))], key=lambda y: int(y[1:])),
    marginalize=True,
    verbose = True
)

RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [19]:
#!pip install wikidata

Defaulting to user installation because normal site-packages is not writeable
  Using cached Wikidata-0.7.0-py3-none-any.whl (29 kB)


In [20]:
client = Client()  # doctest: +SKIP
entity = client.get('Q20145', load=True)
entity, entity.description

(<wikidata.entity.Entity Q20145 'IU'>,
 m'Korea  singer,actress record producer (2017)')

In [21]:
class bcolors:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [22]:
def predict_from_test_set(idx=0):
    print(test_set[str(idx)]['mention_its']+"\n")
    start = test_set[str(idx)]['start_index']
    end = test_set[str(idx)]['end_index']
    mention_id = test_set[str(idx)]['mention_id']
    text = test_set[str(idx)]['source_document']['text']
    text = text[:start]+"[START] "+text[start:end]+" [END]"+text[end:]
    print(f'{text[:start]}{bcolors.BOLD}{text[start:end+len("[START] ")+len(" [END]")]}{bcolors.END}{text[end+len("[START] ")+len(" [END]"):]}\n')
    result = model.sample(
    sentences=[text],
    text_to_id=lambda x: sorted(list(lang_title2wikidataID.get(
        tuple(reversed((x.split(" >> ")[0], x.split(" >> ")[1][:2]))), [None])))[0],
    marginalize=True)
    print(result)
    
    entity = client.get(mention_id, load=True)
    print(f'\nCorrect entity : {mention_id, entity.label, entity.description}\n')

    candidates = [(i['id'], client.get(i['id'], load=True)) for i in result[0]]
    print('Predicted entities:')
    for i, entity in candidates:
        if i is not None:
            print(f'{i, entity.label, entity.description}')

In [23]:
predict_from_test_set(2354)

New South Wales

Bathurst Regional Council, the local government responsible for the city of Bathurst and its surrounds in Central Western [START] New South Wales [END], Australia yesterday revealed it had received a development application for the new Bathurst Base Hospital.The new hospital is to be built behind the current hospital on the same site and is expected to cost the New South Wales government AUD96 million. The Bathurst Hospital will be the first in the Bathurst-Orange-Bloomfield redevelopment project.The new hospital will have 149 beds, up from 85 for the current hospital. The hospital will also feature a mental health unit - previously psychiatric patients had to travel to Orange to the Bloomfield Hospital for treatment.The Bathurst Hospital is expected to have state-of-the art facilities and will share some services with the to be constructed Orange Base Hospital.The Bathurst Regional Council has approved the demolition of 12 buildings on the hospital site for enabling w

/home/petrakov/lalatopola/fairseq/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size


[[{'id': 'Q3224', 'texts': ['New South Wales >> en', 'New South Wales >> de'], 'scores': tensor([-0.0943, -1.9363]), 'score': tensor(-0.2201)}, {'id': None, 'texts': ['New South Welsh English >> en', 'NewSouth Wales >> en'], 'scores': tensor([-1.5939, -1.6029]), 'score': tensor(-3.3681)}, {'id': 'Q1353', 'texts': ['National Capital Territory >> en'], 'scores': tensor([-1.6515]), 'score': tensor(-4.3694)}]]

Correct entity : ('Q3224', m'New South Wales', m'state of Australia')

Predicted entities:
('Q3224', m'New South Wales', m'state of Australia')
('Q1353', m'Delhi', m'Indian metropolis and union territory that includes New Delhi')


# Preliminary results:

Language: **fa** (5 out of 61)

Micro average (k=1): **1.0**

Language: **sr** (105 out of 451)

Micro average (k=1): **0.90476**

Language: **ta** (25 out of 366)

Micro average (k=1): **1.0**

### Macro average (k=1): 0.96825

In [24]:
from tqdm import tqdm
from multiprocessing import Pool

In [27]:
from fairseq.data.data_utils import collate_tokens


def compute_results(idx):
    start = test_set[str(idx)]['start_index']
    end = test_set[str(idx)]['end_index']
    mention_id = test_set[str(idx)]['mention_id']
    text = test_set[str(idx)]['source_document']['text']
    text = text[:start]+"[START] "+text[start:end]+" [END]"+text[end:]
    print(start, end, min(start, len(text)-end))
    #encoder = model.encoder(text)
    #print(encoder)

    result = model.sample(
                    sentences=[text],
                   text_to_id=lambda x: sorted(list(lang_title2wikidataID.get(
       tuple(reversed((x.split(" >> ")[0], x.split(" >> ")[1][:2]))), [None])))[0],
                    marginalize=True)
    candidates = [(i['id'], client.get(i['id'], load=True)) for i in result[0] if i['id'] is not None]
    entity = client.get(mention_id, load=True)
    return {
            "correct": mention_id, 
            "predicted": [i for i, _ in candidates]
               }

In [28]:
compute_results(1)

100 109 100


{'correct': 'Q36678', 'predicted': ['Q36678', 'Q2564150', 'Q7834492']}

In [29]:
lang_title2wikidataID[("ru", "Париж")]

{'Q90'}

In [33]:
def compute_scores(lang='en', k_list=[1, 10, 20, 50, 100]):
    scores = []
    
    for idx in tqdm(langs_idx[lang]):
        scores.append(compute_results(idx))
    return scores

In [34]:
langs_idx = {
    'ar':[], 'en':[], 'tr':[]
}
for idx in test_set:
    el = test_set[idx]
    lang = el['document_id'][:2]
    if lang in langs_idx:
        langs_idx[lang].append(int(idx))

In [35]:
all_langs_micro_avg = []
print('Model F:')
print('on test dataset:\n')

lang2scores = {}

for l in langs_idx:
    lang2scores[l] = compute_scores(l)

Model F:
on test dataset:



  0%|          | 0/1799 [00:00<?, ?it/s]

738 744 14


  0%|          | 1/1799 [00:06<3:04:45,  6.17s/it]

350 357 350


...

  0%|          | 7/1799 [01:04<4:35:35,  9.23s/it]

166 172 166


  0%|          | 8/1799 [01:11<4:25:17,  8.89s/it]

1035 1041 1035


Exception: Size of sample #0 is invalid (=(2949, 0)) since max_positions=(1024, 1024), skip this example with --skip-invalid-size-inputs-valid-test

In [36]:
lang2scores

{}

In [37]:
def micro_avg_lang(scores, lang='en', k_list=[1, 10, 20, 50, 100]):    
    print(f'Language: {lang}')
    results = []
    for k in k_list:
        n = 0
        for i, row in enumerate(scores):
            if scores['correct'] in scores['predicted'][:k]:
                n += 1
        results.append(round(n / total_mentions, 5))
    for i, k in enumerate(k_list):
        print(f'Micro average (k={k}):', results[i])
    return results

In [38]:
for l, scores in lang2scores.items():
    r = micro_avg_lang(scores, l)
    all_langs_micro_avg.append(r)
    print()

k_list=[1, 10, 20, 50, 100]

for i,k in enumerate(k_list):
    print(f'Macro average (k={k}):', round(np.mean(np.array(all_langs_micro_avg)[:,i]), 5))

NameError: name 'np' is not defined